# Задание 2: SQL

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')

import warnings
warnings.filterwarnings('ignore')

import pandahouse as ph

In [2]:
# подключение 
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

## 2.1 Очень усердные ученики

In [3]:
# запрос
q = '''

SELECT count(st_id) AS number_of_good_students
FROM (
    SELECT st_id
    FROM default.peas
    WHERE toStartOfMonth(timest) = (SELECT max(toStartOfMonth(timest))
                                    FROM default.peas)
    GROUP BY st_id
    HAVING SUM(correct) >= 20
    )

'''

good_students = ph.read_clickhouse(query=q, connection=connection_default)
good_students

,number_of_good_students
0,136


## 2.2 Оптимизация воронки

In [4]:
# запрос
q_2 = '''

WITH active_students AS (
    -- добавляем столбцы с информацией об активных студентах
    SELECT 
        st_id,
        if(solved_tasks > 10, 1, 0) AS all_active,
        if(solved_math_tasks >= 2, 1, 0) AS math_active
    FROM (
        -- считаем общее количество правльных зачаний и количество правильных заданий по математике
        SELECT 
            st_id,
            count(correct) AS solved_tasks,
            sumIf(correct, subject = 'Math') AS solved_math_tasks
        FROM default.peas
        GROUP BY st_id
        HAVING correct = 1
        )
)

--
SELECT 
    groups.test_grp AS user_group,
    ROUND(SUM(orders.money) / uniqExact(groups.st_id), 2) AS ARPU,
    ROUND(sumIf(orders.money, active.all_active = 1) / uniqExactIf(groups.st_id, active.all_active = 1), 2) AS ARPAU,
    ROUND(uniqExactIf(groups.st_id, orders.money > 0) / uniqExact(groups.st_id) * 100, 2) AS CR_all,
    ROUND(uniqExactIf(groups.st_id, orders.money > 0 and active.all_active = 1) / 
          uniqExactIf(groups.st_id, active.all_active = 1) * 100, 2) AS CR_active,
    ROUND(uniqExactIf(groups.st_id, orders.money > 0 and active.math_active = 1) / 
          uniqExactIf(groups.st_id, active.math_active = 1) * 100, 2) AS CR_math
FROM default.studs AS groups
LEFT JOIN default.final_project_check AS orders
    ON groups.st_id = orders.st_id
LEFT JOIN active_students AS active
    ON active.st_id = groups.st_id
GROUP BY user_group

'''

metrics = ph.read_clickhouse(query=q_2, connection=connection_default)
metrics

,user_group,ARPU,ARPAU,CR_all,CR_active,CR_math
0,control,4540.98,10393.70,4.92,11.02,10.20
1,pilot,11508.47,29739.58,10.85,26.04,14.29
